In [1]:
import os
os.environ["PATH"] = os.pathsep + "/Users/parksc/Documents/pythonworkspace/gptproject01/ffmpeg/bin"

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import pandas as pd
# from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    chunk_length_s=10,
    stride_length_s=2,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
#sample = dataset[0]["audio"]

sample = "./audio/lsy_audio_2023_58s.mp3"

result = pipe(sample)

start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append((start, end, text))

df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("./audio/lsy_audio_2023_58s.csv", index=False)
display(df)


Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


,start,end,text
0,0.00,6.30,안녕하세요. 이 강의는 GPT API로 챗봇 만들기라는 내용을 다루는 강의입니다.
1,7.16,10.00,GPT API에 대해서 생소하신 분들도 있을 텐데
2,11.00,17.00,우리가 잘 알고 있는 ChatGPT 기능을 이용해서
3,17.00,20.00,우리가 원하는 프로그램을 어떻게 만드는지에 대해서
4,20.00,22.00,이야기할 거에요.
5,22.00,24.00,그래서 이런 강의들이 사실 많이 있습니다.
6,24.00,27.48,그래서 여러가지들이 있는데 이 강의 특징이라고 한다면
7,27.48,29.58,GPT로 명확한 미션을 달성하는
8,29.58,31.62,체포 프로그램을 만드는게 사실
9,31.62,33.00,쉽지는 않은데
